# Problema do Caixeiro-Viajante

O problema do Caixeiro-Viajante trata a seguinte questão: 

_Dada uma lista de cidades e a distância entre elas, qual é a rota mais curta que passa por todas as cidades e retorna à cidade de origem?_

Apesar da curta descrição, este é um dos mais famosos (e complexos) problemas na Ciência da Computação. Encontrar _a rota mais curta_ entre $n$ cidades, com algoritmos não otimizados, gasta um tempo $O(n!)$: a busca pelo caminho mais curto entre 20 cidades exige muito mais que o dobro de recursos gastos para encontrar a melhor rota entre 10 cidades.

Usar busca exaustiva entre todos os caminhos garante que o caminho mais curto será encontrado, mas só é computacionalmente viável para pequenos conjuntos de cidades. Para problemas maiores, técnicas de otimização são necessárias para realizar uma busca inteligente no espaço de estados e encontrar soluções quase-ótimas.

Abaixo, o problema será explicado matemáticamente e uma solução utilizando busca exaustiva será descrita.

### Descrição do problema usando Teoria dos Grafos

O problema do Caixeiro-Viajante pode ser modelado por grafos não direcionados com pesos, de modo que as cidades correspondam às vertices, rotas sejam as arestas, e a distância da rota seja o peso da aresta. A resposta ótima começa e termina no mesmo vértice, após todos os vértices terem sidos visitados pelo menos e somente uma vez. Um modelo baseado em grafos é apresentado na Figura 01.


![Exemplo usando Grafos](imgs/img1.png "Exemplo usando Grafos")
<center style="font-size: 0.8em">Figura 01 - Exemplo de Mapa usando Grafos. Fonte: PIWONSKA, 2011</center>

### Solução utilizando Algoritmos de Busca Não-Informada (i.e. Busca Cega)

#### a) Definição do Escopo do Problema
Para que um problema seja resolvido, é necessário definir seguintes propriedades

- Estados: A descrição do estado corresponde à cidade onde o Caixeiro-Viajante se encontra em um dado momento.
- Estado Inicial: Qualquer uma das cidades.
- Ações Possíveis: Ir à uma das cidades que possuem ligação com a cidade atual.
- Modelo de Transição: A lista das cidades visitadas.
- Teste de Objetivo: Todas as cidades foram visitadas e a cidade atual é a mesma de onde o Caixeiro-Viajante partiu.



####  b) Estados
Será utilizado o mapa apresentado na Figura 01. Para representação computacional, será utilizada uma matriz $10x10$ onde cada elemento $a_{ij}$ indica a distância entre os vértices $i$ e $j$. Para indicar caminhos inexistentes será utilizado o valor $-1$, e para indicar a cidade atual será utilizado o valor $0$.

In [4]:
# Deque => Double-Ended Queue 
# Pode ser utilizada tanto como FIFO quanto como LIFO
from collections import deque

dist_matrix = [
    [ 0,  8, 13, -1, -1, 14, -1,  8, -1, -1], # Cidade 1
    [ 8,  0,  9, 12, -1, -1, -1, -1, -1, 11], # Cidade 2
    [13,  9,  0, -1, 13, 15, -1, -1, -1, -1], # Cidade 3
    [-1, 12, -1,  0, 19, -1, -1, -1, -1, -1], # Cidade 4
    [-1, -1, 13, 19,  0, 15, -1, -1, -1, -1], # Cidade 5
    [14, -1, 15, -1, 15,  0, 22, 18, -1, -1], # Cidade 6
    [-1, -1, -1, -1, -1, 22,  0, -1, 21, -1], # Cidade 7
    [ 8, -1, -1, -1, -1, 18, -1,  0, 10,  8], # Cidade 8
    [-1, -1, -1, -1, -1, -1, 21, 10,  0, 12], # Cidade 9
    [-1, 11, -1, -1, -1, -1, -1,  8, 12,  0]] # Cidade 10

list_of_possible_states = list(range(1, len(dist_matrix) + 1))

print('Lista de Estados Possíveis: %s ' % list_of_possible_states)

Lista de Estados Possíveis: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 


O estado inicial poderá ser qualquer cidade entre 1 e 10

In [5]:
initial_state = 1

if initial_state < 1 or initial_state > len(dist_matrix):
    print('ERRO: Estado Inicial Inválido')
else:
    print('Estado Inicial: Cidade %s' % initial_state)

Estado Inicial: Cidade 1


#### c) Estruturas de Dados e Processamento
O primeiro passo para se elaborar uma solução utilizando Busca Não-Informada é criar um modelo de estrutura de dados para manter o controle da árvore de busca que será construída.

##### c.1) Nós
O Modelo de Nó descrito no livro _Artificial Intelligente: A Modern Approach_, faz uso de três atributos:
- Estado: O estado no espaço de estados que gerou este nó
- Pai: O nó da árvore que gerou esse nó
- Ação: A ação que foi aplicada para gerar esse nó
- Custo do Caminho: O custo do caminho inicial até este nó

Para descrever um nó, será criada uma classe com todas essas características.

In [6]:
class Node:
    """ Classe para representação de Nós """
    
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __repr__(self):
        if self.parent:
            return 'Node<State: %s, Parent: %s, Action: %s, Cost: %s>' % (self.state, self.parent.state, self.action, self.path_cost)
        else:
            return 'Node<State: %s, Parent: %s, Action: %s, Cost: %s>\n' % (self.state, 'None', self.action, self.path_cost)

Dada a estrutura de um Nó, é fácil criar um método que retorne os componentes para um Nó-Filho. O método _child_\__node_ que recebe um Nó-Pai, uma ação (que é um inteiro, indicando a cidade de destino) e o custo da ação atual, para então retornar o Nó-Filho resultante.

In [7]:
def child_node(problem, parent, action):
    """ Método que retorna um Nó-Filho à partir de um Nó-Pai"""
    
    state = problem.result(parent.state, action)
    path_cost = parent.path_cost + problem.step_cost(parent.state, action)
    
    child = Node(state, parent, action, path_cost) 
    
    return child

##### c.2) O problema
São necessárias algumas variáveis de controle, além de métodos específicos ao problema. Para isso, será criada uma classe chamada _TravellingSalesman_ que abstrairá essas estruturas, permitindo que o Algoritmo de Busca seja genérico, e não específico ao problema: 
- Fila: A sequência de ações realizadas, inicialmente vazia.
- Estado: O estado atual.
- Teste de Objetivo: Verifica se o objetivo foi cumprido.
- Ações Possíveis: Retorna uma lista com ações possíveis dado o estado atual.

__Atenção__: Por não ser uma busca otimizada, não será verificado se todas as cidades foram visitadas SOMENTE uma vez.

In [16]:
class TravellingSalesperson:
    """ Classe para organização de dados do problema """
    
    def __init__(self, initial_state):
        self.initial_state = initial_state
    
    def goal_test(self, node):
        """ Retorna verdadeiro caso o objetivo tenha sido alcançado """
        
        seq = self.solution(node)
        
        return all(elem in seq for elem in list_of_possible_states) and seq[len(seq)-1] == initial_state
    
    def actions(self, cur_state):
        """ Retorna lista de cidades que podem ser acessadas à partir da cidade atual """
               
        return [state+1 for state, distance in filter(lambda x: x[1] > 0, enumerate(dist_matrix[cur_state-1]))]
    
    def solution(self, node):
        """ Retorna os passos executados para se chegar no estado do nó """
        seq = deque()
        
        while not node.parent == None:
            seq.appendleft(node.action)
            node = node.parent
            
        return seq
    
    def result(self, state, action):
        """ Retorna o estado caso seja realizada uma ação """
        return action
    
    def step_cost(self, state, action):
        """ Retorna o custo de realizar uma ação dado um estado """
        return dist_matrix[state-1][action-1]

salesperson = TravellingSalesperson(initial_state)

### Busca Não-Informada

O termo Busca Não-Informada (ou Busca Cega) indica que não são utilizadas informações além daquelas apresentadas pelo problema. Tudo que o agente pode fazer é verificar se o objetivo foi ou não atingido. 

Existem dois algoritmos principais de Busca Não-Informada: Busca em Largura e Busca em Profundidade. Na Busca em Largura, todos os nós possíveis são expandidos igualmente, e verifica-se se o resultado foi atingido. Na Busca em Profundidade, cada nó é expandido individualmente até que alguma condição de falha seja atingida, e só então o próximo nó aberto (fronteira) é avaliado.

#### Criação do Nó Inicial
Independente do algoritmo de busca não-informada utilizado, este começa à partir de um Nó Inicial que contém as informações do estado inicial.

__Atenção__: Neste problema, a resolução depende de todos os estados passados, portanto o estado armazenado no Nó corresponde ao conjunto dos estados passados, para simplificar o algoritmo de verificação. Outra possibilidade, apresentada no livro _Artificial Intelligence: A Modern Approach_ para avaliar se o objetivo foi atingido, seria analisar recursivamente os Nós da árvore, o que diminuiria o uso de memória para armazenamento da árvore mas aumentaria o tempo de processamento.

#### Busca em Largura (_Breadth-First Search \[BFS\]_)
Esta é uma estratégia simples onde o nó inicial é expandido primeiro e todos os seus sucessores diretos são expandidos em seguida. 

In [17]:
def breadth_first_search(problem):
    node = Node(problem.initial_state, None, None, 0)

    if salesperson.goal_test(node):
        return solution(node)

    frontier = deque([node])
    explored = deque()
    
    while True:
        if len(frontier) == 0:
            return 'Failure'
        
        node = frontier.popleft()
        explored.append(node.state)
        
        for action in problem.actions(node.state):
            child = child_node(problem, node, action)
            
            if not child.state in problem.solution(child.parent):  
                if problem.goal_test(child):
                    return problem.solution(child)
                
                frontier.append(child)
        

In [18]:
print('SOLUTION: ', breadth_first_search(salesperson))

SOLUTION:  deque([2, 4, 5, 3, 6, 7, 9, 10, 8, 1])


#### Busca em Profundidade (Depht-First Search [DFS])



Nesta estratégia de busca, o nó é expandido até que chegue ao fim (ou atinga uma profundidade pré-determinada). A única mudança do algoritmo BFS é a estrutura de dados, que neste caso armazena os nós em uma pilha.

In [19]:
def depth_first_search(problem):
    node = Node(problem.initial_state, None, None, 0)

    if salesperson.goal_test(node):
        return solution(node)

    frontier = deque([node])
    explored = deque()
    
    while True:
        if len(frontier) == 0:
            return 'Failure'
        
        node = frontier.popleft()
        explored.append(node.state)
        
        for action in problem.actions(node.state):
            child = child_node(problem, node, action)
            
            if not child.state in problem.solution(child.parent):
                if problem.goal_test(child):
                    return problem.solution(child)
                
                frontier.appendleft(child)
        

In [20]:
print('SOLUTION: ', depth_first_search(salesperson))

SOLUTION:  deque([8, 10, 9, 7, 6, 5, 4, 2, 3, 1])


#### Busca de Custo Uniforme (Uniform-Coast Search [UCS])

Nesta estratégia de busca, o nó com menor caminho é o primeiro a ser expandido. os nós são adicionados na estrutura de dados de acordo com seu custo.

In [21]:
def insertOrdered(array, node):
    """ Adiciona um nós ao deque em ordem de custo """
    pos = len(array)
    
    for index, cur_node in enumerate(array): 
        if node.path_cost < cur_node.path_cost:
            pos = index
            break;

    array.insert(pos, node)

In [22]:
def uniform_cost_search(problem):
    node = Node(problem.initial_state, None, None, 0)

    if salesperson.goal_test(node):
        return solution(node)

    frontier = deque([node])
    
    while True:
        if len(frontier) == 0:
            return 'Failure'
        
        node = frontier.popleft()

        for action in problem.actions(node.state):
            child = child_node(problem, node, action)
            
            if not child.state in problem.solution(child.parent):
                if problem.goal_test(child):
                    return problem.solution(child)
                
                insertOrdered(frontier, child)
        

In [23]:
print('SOLUTION: ', uniform_cost_search(salesperson))

SOLUTION:  deque([8, 10, 9, 7, 6, 5, 4, 2, 3, 1])
